In [56]:
from apify_client import ApifyClient
import datetime
import json
import pandas as pd
import csv
from bs4 import BeautifulSoup
from selenium import webdriver
import time
import math

In [57]:
client = ApifyClient("apify_api_LJB3wsOd9H1iYEdfnBflCoSc8nO9GG0zX6TS")

In [58]:
def crawl_googlesearch_by_apify(input, client, num, time_now):
    
    
    run_input = {
        "queries": f"{input}",
        "lang": "vi",
        "maxPagesPerQuery": math.ceil(int(num+30)/10),
        "resultsPerPage": 10,
        "countryCode": "",
        "customDataFunction": """async ({ input, $, request, response, html }) => {
            return {
                pageTitle: $('title').text(),
            };
        };""",
    }

    run = client.actor("apify/google-search-scraper").call(run_input=run_input)

    results = []
    for item in client.dataset(run["defaultDatasetId"]).iterate_items():
        results.append(item)

    with open(f"data/googlesearch/{time_now}_{input}_apify.json", "w") as file:
        json.dump(results, file)


In [59]:
def crawl_googlesearch_by_selenium(key, num_results, time_now):
    key = key.replace(" ", "+")
    filename = f'./data/googlesearch/{time_now}_{key}_selenium.csv'

    fields = ['title', 'link', 'description']
    driver = webdriver.Chrome()
    with open(filename, 'w', newline='', encoding='utf-8') as csvfile:
        writer = csv.DictWriter(csvfile, fieldnames=fields)
        writer.writeheader()
        result = 0
        for i in range(math.ceil((num_results + 50) / 10)):
            link = f'https://www.google.com/search?q={key}&start={i*10}'
            driver.get(link)
            driver.implicitly_wait(5)
            html = driver.page_source
            soup = BeautifulSoup(html, 'html.parser')
            h3_tags = soup.find_all('h3', class_='LC20lb MBeuO DKV0Md')
            divs = soup.find_all('div', class_='yuRUbf')
            div_des = soup.find_all('div', class_='VwiC3b yXK7lf MUxGbd yDYNvb lyLwlc lEBKkf')
            for h3_tag, div, div_des in zip(h3_tags, divs, div_des):
                title = h3_tag.text
                a_tag = div.find('a')
                href = a_tag.get('href')
                descriptions = div_des.find_all('span')
                description = ''
                for des in descriptions:
                    description = description+ des.text
                writer.writerow({'title': title, 'link': href, 'description' : description})
                result += 1
                if result == num_results:
                    break
            if result == num_results:
                break

In [60]:
key = input("Nhập từ khóa bạn muốn tìm kiếm: ")
num_results = int(input("Nhập số lượng bài viết bạn muốn tìm kiếm: "))
current_time = datetime.datetime.now()
time_now = current_time.strftime("%Y-%m-%d_%H-%M")
start_time = time.time()
crawl_googlesearch_by_apify(key, client, num_results, time_now)
end_time = time.time()
elapsed_time = end_time - start_time
print(f"Time to search {num_results} results by apify: {elapsed_time}")
start_time = time.time()
crawl_googlesearch_by_selenium(key, num_results, time_now)
end_time = time.time()
elapsed_time = end_time - start_time
print(f"Time to search {num_results} results by selenium: {elapsed_time}")

Time to search 66 results by apify: 16.039082765579224
Time to search 66 results by selenium: 14.865723371505737
